# ISyE 6501 - HW11: Optimization

## Problem Set 15

### 15.2.1: Optimizing the cheapest diet that maintains min and max dietary constraints

##### Check Python version, import necessary packages, load data

In [1]:
from platform import python_version

print(python_version())

3.8.3


In [2]:
import pandas as pd
import numpy as np
from pulp import *
#!pip install pulp

In [3]:
df = pd.read_excel('C:/Users/Clayton/Desktop/Georgia Tech/ISYE_6501/hw11/data 15.2/diet.xls')

##### Dataframe rebuild, so we isolate the variables, and the min-max constraints for later use

In [4]:
variable_df = df.loc[0:63, df.columns != 'Serving Size']
constraint_df = df.iloc[65:67, 3:]
min_con = constraint_df.iloc[[0,]]
max_con = constraint_df.iloc[[1,]]

In [5]:
variable_df
constraint_df
min_con
max_con
variable_df

,Foods,Price/ Serving,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Neweng Clamchwd,0.75,175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6


##### Build a bunch of food-nutrient dictionaries. Probably should have looped this.

In [13]:
food_items = list(variable_df['Foods'])
food_items
costs = dict(zip(food_items,variable_df['Price/ Serving']))
calories = dict(zip(food_items,variable_df['Calories']))
cholesterol = dict(zip(food_items,variable_df['Cholesterol mg']))
fat = dict(zip(food_items,variable_df['Total_Fat g']))
sodium = dict(zip(food_items,variable_df['Sodium mg']))
carbs = dict(zip(food_items,variable_df['Carbohydrates g']))
fiber = dict(zip(food_items,variable_df['Dietary_Fiber g']))
protein = dict(zip(food_items,variable_df['Protein g']))
vitamin_a = dict(zip(food_items,variable_df['Vit_A IU']))
vitamin_c = dict(zip(food_items,variable_df['Vit_C IU']))
calcium = dict(zip(food_items,variable_df['Calcium mg']))
iron = dict(zip(food_items,variable_df['Iron mg']))
servings = dict(zip(food_items,df['Serving Size']))
nutrients = ['calories', 'cholesterol', 'fat', 'sodium', 'carbs', 'fiber', 'protien', 'vitamin_a', 'vitamin_c', 'calcium', 'iron']
nutrient_dicts = [calories, cholesterol, fat, sodium, carbs, fiber, protein, vitamin_a, vitamin_c, calcium, iron]

##### Construct the Food Variable LP dict as a continuous zero-floor var

In [7]:
food_vars = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')
#food_vars

##### build the optimization problem

In [8]:
prob = LpProblem("Diet-Cost Optimization",LpMinimize)
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


##### Loop through food-nutrient constraints

In [9]:
j = 0
for i in nutrient_dicts:
    prob += lpSum([i[f] * food_vars[f] for f in food_items]) >= float(min_con.iloc[:,j])
    prob += lpSum([i[f] * food_vars[f] for f in food_items]) <= float(max_con.iloc[:,j])
    j += 1

##### Solve and display the optimization problem

In [10]:
prob.solve()

1

In [26]:
for i in prob.variables():
    if i.varValue>0:
        print(i.name, "=", i.varValue, "servings.")

Food_Celery,_Raw = 52.64371 servings.
Food_Frozen_Broccoli = 0.25960653 servings.
Food_Lettuce,Iceberg,Raw = 63.988506 servings.
Food_Oranges = 2.2929389 servings.
Food_Poached_Eggs = 0.14184397 servings.
Food_Popcorn,Air_Popped = 13.869322 servings.


In [27]:
obj = value(prob.objective)
print("Cost of above diet is: ${}".format(round(obj,2)))

Cost of above diet is: $4.34


### 15.2.2: Adding three additional complexities

##### Two new variables: a food amounts variable (similar to the one used above), and a binary toggle variable.

In [30]:
food_amounts_variable = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')
food_binary_variable = LpVariable.dicts("Binary",food_items,0,1,cat='Integer')

In [31]:
prob2 = LpProblem("Diet-Cost Optimization with Additional Complexities",LpMinimize)
prob2 += lpSum([costs[i]*food_amounts_variable[i] for i in food_items])

##### Keep the nutrient constraints the same as above.

In [32]:
j = 0
for i in nutrient_dicts:
    prob2 += lpSum([i[f] * food_amounts_variable[f] for f in food_items]) >= float(min_con.iloc[:,j])
    prob2 += lpSum([i[f] * food_amounts_variable[f] for f in food_items]) <= float(max_con.iloc[:,j])
    j += 1

##### Binary inclusion plus a 1/10th serving min

In [33]:
for f in food_items:
    prob2 += food_amounts_variable[f] >= food_binary_variable[f]*0.1
    prob2 += food_amounts_variable[f] <= food_binary_variable[f]*100000

##### Include NEITHER or AT MOST ONE of Frozen Broccoli and Raw Celery, I.E. the sum of the binaries must be less than or eqaul to 1.

In [35]:
prob2 += food_binary_variable['Frozen Broccoli']+food_binary_variable['Celery, Raw']<=1

##### Generate an indicator variable for which foods are PROTEINS, and make a dictionary out of them

In [36]:
proteins = ['Tofu', 'Roasted Chicken', 'Poached Eggs', 'Scrambled Eggs', 'Bologna,Turkey', \
            'Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', \
            'Hamburger W/Toppings', 'Hotdog, Plain', 'Pork', 'Sardines in Oil', 'White Tuna in Water']

protein_df = pd.DataFrame(variable_df.loc[:,'Foods'])
protein_df['Protein'] = protein_df['Foods'].apply(lambda x: 1 if x in proteins else 0)
proteins_dict = dict(zip(protein_df.Foods, protein_df.Protein))

##### The sum of protein binaries in the final food alotment must be at least 3.

In [37]:
prob2 += lpSum([proteins_dict[f]*food_binary_variable[f] for f in food_items]) >= 3

##### Solve the new Optimization, and show the updated meal and costs.

In [38]:
prob2.solve()

1

In [44]:
for i in prob2.variables():
    if i.varValue>0 and i.name.startswith('Food_'):
        print(i.name, "=", i.varValue)

Food_Celery,_Raw = 42.399358
Food_Kielbasa,Prk = 0.1
Food_Lettuce,Iceberg,Raw = 82.802586
Food_Oranges = 3.0771841
Food_Peanut_Butter = 1.9429716
Food_Poached_Eggs = 0.1
Food_Popcorn,Air_Popped = 13.223294
Food_Scrambled_Eggs = 0.1


In [40]:
obj = value(prob2.objective)
print("The total cost of this balanced diet is: ${}".format(round(obj,2)))

The total cost of this balanced diet is: $4.51
